In [ ]:
import numpy as np
import pandas as pd
TARGET_LABEL = 'transient_type'

def generate_data(original, num_datapoints):
    data = []
    for i in range(num_datapoints):
        noise = np.random.normal(loc=0, scale=0.1, size=3)
        data.append(original + noise)
        
        
    return np.array(data)
num_type1 = 20
num_type2 = 10
c1 = generate_data([0.2, 0.3, .9], num_type1)
c2 = generate_data([0.3, 0.1, .8], num_type2)
data_X = np.concatenate((c1,c2))
fake_X = pd.DataFrame(data_X, columns = ['f1', 'f2', 'f3'])
fake_Y = pd.DataFrame(["Ia"]*num_type1 + ["CC"]*num_type2 , columns = [TARGET_LABEL])

fake_data = [fake_X, fake_Y]

In [ ]:
from models.multi_model.multi_model import MultiModel
%matplotlib inline 


model = MultiModel( #cols = ['W1_mag'],
#         data = fake_data,
         class_labels =["Ia", "CC",  "TDE", "GRB", "FRB", "GW", "Kilonova"],
         subsample = 100,
        )

model.run_model()


In [ ]:

import warnings
warnings.filterwarnings('ignore')
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)

from models.ind_model.ind_model import IndModel
%matplotlib inline 

import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)



model = IndModel( #cols = ['W1_mag'],
#           data = fake_data,
         class_labels =["Ia", "CC",  "TDE", "GRB", "FRB", "GW", "Kilonova"],
         subsample = 500,
        )
model.run_model()
